### 데이터를 고정-길이 필드로 변환(struct)  

읽고 쓸 필요가 있는 신규 데이터 파일을 처음부터 만든다면 피클링 인터페이스가 가장 쉽다는 것을 알게 될 것이다. 반면 파이썬으로 만들지 않은 기존 바이너리 파일을 다루려면 다양한 크기의 정수와 부동소수점 숫자, 문자열을 읽고 쓸 저수준 방법이 필요하다. 지난 절에서 살펴본 것과 같이 한 번에 읽고 쓸 바이트를 하나씩 읽는 것이 가능하더라도, 이 방법은 이식성이 없으며 데이터 사용을 어렵게 만든다.  

struct 패키지는 바이트 문자열 안에 익숙한 내장 타입을 패킹하거나 언패킹 하는 데 도움을 주는 도구다. 다음과 같이 여러 메서드를 포함하고 있다.  

import struct
바이트_문자열 = struct.pack(포맷_문자열, v1, v2, v3...)  
v1, v2, v3... = struct.unpack(포맷_문자열, 바이트_문자열)  
struct.calcsize(포맷_문자열)  

* struct.pack 함수는 포맷 문자열 1개 이상의 값을 인수로 받는다. 이 함수는 바이너리 파일에 쓸 수 있는 바이트 문자열을 반환한다. 
* struct.unpack 함수는 이와 반대다. 타입 바이트의 문자열을 인수로 받아서 튜플로 여러 값을 반환한다. 값의 타입은 '포맷_문자열' 인수로 결정된다
* struct.calcsize 함수는 주어진 '포맷_문자열'이 요구하는 바이트를 반환한다. '바이트_문자열'이 타입 바이트인 반면, '포맷_문자열'은 평범한 파이썬 문자열이다.  

패킹과 언패킹을 위한 일반 적인 데이터 포맷  
* c, bytes, 1
* ?, bool, 1
* h, int, 2
* H, int, 2
* l, int, 4
* L, int, 4
* q, int, 8
* Q, int, 8
* f, float, 4
* d, float, 8
* ints, str, int길이
* p, 파스칼 문자열 타입  

struct 패키지를 사용하는 바이너리 파일을 사용하려면 다음 단계를 거쳐야 한다.  

* 파일을 바이너리 쓰기 모드('wb')로 연다
* 문자열을 쓰려면 먼저 문자열 클래스의 encode 메서드로 문자열을 bytes 문자열로 변환한다
* struct.pack 함수를 사용하여 모든 데이터의 bytes 문자열을 패킹한다. 위의 표에 나열된 데이터-포맷 지시자를 1개 이상 사용해야 한다. 가령 'h'는 16-비트 정수를 의미한다. 입력할 모든 문자열은 바로 앞 단계에서 설명했듯이 이미 변환이 되어 있어야 한다
* 최종적으로 바이트 문자열을 파일 객체의 write 메서드를 사용하여 파일에 쓴다  

struct 패키지를 사용하여 바이너리 파일을 읽는 절차도 비슷하다  
* 파일을 바이너리 읽기 모드('wb')로 연다 
* 바이트의 문자열을 읽는다. 반드시 읽을 정확한 바이트 개수를 명시해야 하기 때문에 읽기에 앞서 데이터 사이즈를 알 필요가 있다. 데이터 사이즈는 struct.calcsize 함수를 위의 표에서 나열한 데이터-포맷 문자열과 함께 실행하면 알 수 있다.  

bss = f.read(struct.calcsize('h'))  

* struct.unpack 함수를 사용하여 바이트 문자열 값들을 튜플로 언팩한다. 결과가 튜플이기 때문에 각 항목에 접근하려면 색인을 사용해야 한다. 항목이 하나더라도 마찬가지다 예시를 살펴보자.  

tup = struct.unpack('h', bss)
return tup[0]  

* 3번째 단계에서 일반 파이썬 문자열로 대입되는 바이트 문자열을 읽었다면, bytes 클래스의 decode 메서드를 사용하여 변환한 문자열을 읽을 수 있다.  

이런 기술들은 바이트의 저수준 위치를 다루기 때문에 빅 엔디안과 리틀 엔디안의 선택, 패딩을 처리하기 위한 특별한 고려 사항이 있다. 하지만 먼저 다음 몇개의 하위 섹션에서 구체적인 문제점을 살펴보겠다.  

* 한 번에 하나의 숫자 읽기/쓰기
* 한 번에 여러 숫자 읽기/쓰기
* 고정-길이 문자열 읽기/쓰기
* 변수-길이 문자열 읽기/쓰기
* 여러 타입이 섞인 데이터 읽기/쓰기  

##### 한 번에 하나의 숫자 읽기/쓰기  

패킹이 된 하나의 숫자(이번 예시에서는 정수)를 읽고 쓸 때 발생하는 이슈는 매우 간단하지만, 이 튜플을 읽으려면 단 하나의 항목만을 가지고 있더라도 색인을 사용하여 접근해야 한다는 것을 기억하자  

In [3]:
from struct import pack, unpack, calcsize

def write_num(fname, n):
    with open(fname, 'wb') as f:
        bss = pack('h', n)
        f.write(bss)
        
def read_num(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('h'))
        t = struct.unpack('h', bss)
        return t[0]
    
write_num('silly.dat', 125)
print(read_num('silly.dat'))

125


이 코드 정의문에는 파일에서 읽거나 쓸 정수가 short-integer(16-비트)포맷이라고 가정한다. 

##### 한 번에 여러 숫자 읽기/쓰기  

이 문제는 앞 절에서 다룬 것과 비슷하다. 하지만 1개 이상의 숫자를 반환하기 때문에 가장 간단한 해결책은 read 함수를 사용하여 반환값을 튜플로 반환하는 것이다. 다양한 예시를 다루기 위해 이번 예시는 부동솟점 숫자 3개를 사용했다. 

In [7]:
from struct import pack, unpack, calcsize

def write_floats(fname, x, y, z):
    with open(fname, 'wb') as f:
        bss = pack('fff', x, y, z)
        f.write(bss)
        
def read_floats(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('fff'))
        return unpack('fff', bss)

write_floats('silly.dat', 1, 2, 3.14)
x, y, z = read_floats('silly.dat')
print(x, y, z, sep='\n')

1.0
2.0
3.140000104904175


이번 예시의 'fff'는 '3f'로 교체할 수 있다는 것을 명심하자. 위 예시에서 는 부동소수점 숫자 3개를 한 번에 읽고 쓰는 방법을 보여준다. 값이 3개 출력된다. 마지막 항목은 분명한 반올림 에러다. 

##### 고정-길이 문자열 읽기/쓰기  

여러분이 다루기 가장 쉽다고 생각하는 문자열을 바이너리로 저장하면 예상치 못한 문제가 생긴다.  
첫째, 파이썬 문자열이 단일-바이트 포맷을 사용한다고 가정할 수 없기 때문에 문자열을 바이트 문자열로 인코드 혹은 디코드 할 필요가 있다.  
둘째, 문자열 길이는 다양하기 때문에 바이너리 연산을 사용하려면 얼마나 많은 문자를 읽거나 쓸지 제시해야한다. 이 문제는 결코 단순한 문제가 아니며 한 가지 해결책은 함수를 호출할 때 읽거나 쓸 문자의 개수를 명시하는 것이다. 

In [9]:
from struct import pack, unpack, calcsize

def write_fixed_str(fname, n, s):
    with open(fname, 'wb') as f:
        bss = pack(str(n) + 's', s.encode('utf-8'))
        f.write(bss)
        
def read_fixed_str(fname, n):
    with open(fname, 'rb') as f:
        bss = f.read(n)
        return bss.decode('utf-8')
    
write_fixed_str('king.d', 13, "I'm Henry the VIII I am!")
print(read_fixed_str('king.d', 13))

I'm Henry the


이 한 쌍의 함수는 문자열을 읽거나 쓰는 시점보다 앞서서 문자열의 길이를 서로 알아야 한다. 이 길이는 완벽하게 일치해야만 한다.  

write_fixed_str 함수가 자동으로 문자열 (추가 null 바이트 포함)을 잘라 내거나 추가하는 pack 함수를 호출하여 길이가 n 에 이르게 한다.  

두 번째 줄은 13개의 문자만 읽는데, 읽을 문자의 개수가 13자 뿐이기 때문이다. 

##### 변수-길이 문자열 읽기/쓰기  

이 절에서 다룰 방법은 지난 절에서 다룬 방법보다 더욱 정교하다. 함수의 사용자가 인수로 문자를 주면 적절한 숫자의 바이트를 스거나 읽기 때문이다 

In [12]:
from struct import pack, unpack, calcsize

def write_var_str(fname, s):
    with open(fname, 'wb') as f:
        n = len(s)
        fmt = 'h' + str(n) + 's'
        bss = pack(fmt, n, s.encode('utf-8'))
        f.write(bss)
        
def read_var_str(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('h'))
        n = unpack('h', bss)[0]
        bss = f.read(n)
        return bss.decode('utf-8')

write_var_str('silly.dat', "I'm Henry the VIII I am!")
print(read_var_str('silly.dat'))

I'm Henry the VIII I am!


write_var_str 함수는 묘책을 갖고 있다. 먼저 h 숫자 s 형식의 문자열 포맷 지시자를 만든다. 다음 예시에서 이 포맷 지시자는 h24s 동적으로 설정되며, "24개의 문자로 구성된 문자열에 표기한 정수를 쓴다." 라는 의미를 갖는다  

read_var_str 함수는 정수 24를 읽고, 정확하게 얼마나 많은 바이트를 읽을지 결정한다. 최종적으로 이 바이트들은 표준 파이썬 텍스트 문자열로 디코드 된다.  



##### 문자열과 숫자를 함께 읽기/쓰기  

다음 한 쌍의 함수는 길이가 9인 문자열, 길이가 10인 문자열과 부동소수점 숫자로 구성된 레코드를 읽고 쓴다. 

In [15]:
from struct import pack, unpack, calcsize 

def write_rec(fname, name, addr, rating):
    with open(fname, 'wb') as f:
        bname = name.encode('utf-8')
        baddr = addr.encode('utf-8')
        bss = pack('9s10sf', bname, baddr, rating)
        f.write(bss)
        
def read_rec(fname):
    with open(fname, 'rb') as f:
        bss = f.read(calcsize('9s10sf'))
        bname, baddr, rating = unpack('9s10sf', bss)
        name = bname.decode('utf-8').rsplit('\x00')
        addr = baddr.decode('utf-8').rsplit('\x00')
        return name, addr, rating
    
write_rec('goofy.dat', 'Cleo', 'Main St.', 5.0)
print(read_rec('goofy.dat'))

(['Cleo', '', '', '', '', ''], ['Main St.', '', ''], 5.0)


##### 저수준 상세; 빅 엔디안 vs 리틀 엔디안  

1개가 아닌 3개의 정수를 쓰는 코드를 생각해 보자.  

In [18]:
import struct  

with open('junk.dat', 'wb') as f:
    bstr = struct.pack('hhh', 1, 2, 100)
    datalen = f.write(bstr)

실제로 쓰기를 할 바이트 개수를 저장한 변수 datalen을 구하면 값이 6이라는 것을 찾게 될 것이다. 이 숫자는 calcsize로도 구할 수 있다. 숫자 1, 2 ,100은 각각 2-바이트 정수(포맷 h) 이기 때문이다. 파이썬 자체에서 이런 정수들은 훨씬 더 많은 공간을 차지한다  

유사한 코드를 사용하여 추후 파일에서 값을 다시 읽을 수도 있다. 

In [19]:
with open('junk.dat', 'rb') as f:
    bstr = f.read(struct.calcsize('hhh'))
    a, b, c = struct.unpack('hhh', bstr)
    print(a, b, c)

1 2 100


다은 예시는 조금 더 흥미로운 사례를 다룬다. 2개의 정수와 long 정수가 뒤따르는 사례다. 코드를 살펴보고 나서, 이 코드에서 다루는 복잡한 문제를 다룰 것이다. 

In [23]:
with open('juck.dat', 'wb') as f:
    bstr = struct.pack('hhl', 1, 2, 100)
    datalen = f.write(bstr)
    
with open('juck.dat', 'rb') as f:
    bstr = f.read(struct.calcsize('hhl'))
    a, b, c = struct.unpack('hhl', bstr)
    
print(bstr)
    

b'\x01\x00\x02\x00\x00\x00\x00\x00d\x00\x00\x00\x00\x00\x00\x00'


이 예시는 (hhh 대신 hhl 포맷을 사용하는 경우를 제외하고) 앞서 살펴본 예시와 같이 동작하지만 바이트 문자열인 bstr을 출력한다. 이 부분이 매우 중요하다.  

주목해야 할 사항은 다음과 같다.  

* 바이트 배치를 자세히 살펴보면, 이 예시와 지난 코드(바이트 문자열을 찾는 예시)에서 리틀-엔디안 바이트 배치를 사용하고 있다는 것을 알게 될 것이다. 정수 필드 같은 경우 최하위 숫자가 제일 먼저 위치한다. 필자의 시스템은 모토롤라 프로세서를 사용하는 mac OS 이기 때문에 이 방식을 사용하고 있다. 모든 프로세스는 각각 서로 다른 표준을 사용할 수 있다. 
* long 정수(100 혹은 16진수 값 d)는 반드시 32-비트 경계에서 시작하기 때문에 두 번째 인수와 세 번째 인수 사이에 2바이트 패딩이 위치하게 된다. 이전 섹션의 끝에서 이 이슈에 대해서 언급했었다.  

여러분의 시스템이 리틀-엔디안을 사용하고 있는데, 프로그램이 파일을 빅-엔디안으로 읽으면 문제가 발생할 것이다. 반대도 마찬가지다. struct 함수는 포맷 문자열의 시작 부분에 빅 엔디안 혹은 리틀 엔디안을 명시할 수 있는 방법을 제공한다. 밑의 표는 바이너리 데이터를 제어하기 위한 저수준 모드를 나열하고 있다.  

* < : 리틀 엔디안 
* \>: 빅 엔디안
* @ : 로컬 머신 네이티브